In [5]:
import pygame, sys
from pygame.locals import *
import random, time

# Initialize pygame
pygame.init()

# Setting up FPS (Frames Per Second)
FPS = 60
FramePerSec = pygame.time.Clock()

# Defining colors
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 223, 0)
SILVER = (0, 0, 0)
GOLD = (255, 215, 0)

# Screen dimensions and game variables
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 5
SCORE = 0
COINS_COLLECTED = 0
speed_upp = 5  # Number of coins needed to increase enemy speed
COIN_TYPES = [
    {"color": YELLOW, "weight": 1, "radius": 10},  # Regular coin
    {"color": SILVER, "weight": 2, "radius": 12},  # Black coin 
    {"color": GOLD, "weight": 3, "radius": 14}     # Gold coin 
]

# Setting up fonts
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, BLACK)

# Loading background image
background = pygame.image.load("AnimatedStreet.png")

# Setting up display
DISPLAYSURF = pygame.display.set_mode((400,600))
DISPLAYSURF.fill(WHITE)
pygame.display.set_caption("GTA VI")

class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH-40), 0)  

    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)
        if self.rect.top > SCREEN_HEIGHT:
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Player.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)
        
    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if self.rect.left > 0 and pressed_keys[K_LEFT]:
            self.rect.move_ip(-5, 0)
        if self.rect.right < SCREEN_WIDTH and pressed_keys[K_RIGHT]:        
            self.rect.move_ip(5, 0)

class Coin:
    def __init__(self):
        self.type = random.choice(COIN_TYPES)
        self.color = self.type["color"]
        self.weight = self.type["weight"]
        self.radius = self.type["radius"]
        self.x = random.randint(40, SCREEN_WIDTH - 40)
        self.y = random.randint(0, SCREEN_HEIGHT // 2)
    
    def move(self):
        self.y += SPEED
        if self.y > SCREEN_HEIGHT:
            self.respawn()
    
    def respawn(self):
        self.__init__()
    
    def draw(self, surface):
        pygame.draw.circle(surface, self.color, (self.x, self.y), self.radius)

# Initialize player and enemy
P1 = Player()
E1 = Enemy()


enemies = pygame.sprite.Group()
enemies.add(E1)
all_sprites = pygame.sprite.Group()
all_sprites.add(P1, E1)

# Create coins
coins = [Coin() for _ in range(3)]  # Start with 3 coins on screen

# Adding a speed increase event
INC_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INC_SPEED, 1000)

# Game loop
while True:
    for event in pygame.event.get():
        if event.type == INC_SPEED:
            SPEED += 0.2  # Slightly slower speed increase     
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    DISPLAYSURF.blit(background, (0,0))
    
    # Displaying scores and coins collected
    scores = font_small.render(f"Score: {SCORE}", True, BLACK)
    DISPLAYSURF.blit(scores, (10,10))
    coins_display = font_small.render(f"Coins: {COINS_COLLECTED}", True, BLACK)
    DISPLAYSURF.blit(coins_display, (SCREEN_WIDTH - 120, 10))

    # Moving and drawing sprites
    for entity in all_sprites:
        DISPLAYSURF.blit(entity.image, entity.rect)
        entity.move()

    # Moving and drawing coins
    for coin in coins:
        coin.move()
        coin.draw(DISPLAYSURF)

    # Checking collision with enemy
    if pygame.sprite.spritecollideany(P1, enemies):
        pygame.mixer.Sound('crash.wav').play()
        time.sleep(0.5)
        
        DISPLAYSURF.fill(RED)
        DISPLAYSURF.blit(game_over, (30,250))
        
        pygame.display.update()
        for entity in all_sprites:
            entity.kill()
        time.sleep(2)
        pygame.quit()
        sys.exit()
    
    # Checking collision with coins
    for coin in coins:
        if (abs(P1.rect.centerx - coin.x) < 20 and 
            abs(P1.rect.centery - coin.y) < 20):
            COINS_COLLECTED += coin.weight
            coin.respawn()
            
            # Increase enemy speed every speed_upp coins
            if COINS_COLLECTED % speed_upp == 0:
                SPEED += 1  # More significant speed boost
                print(f"Speed increased to {SPEED} after {COINS_COLLECTED} coins")
    
    pygame.display.update()
    FramePerSec.tick(FPS)

Speed increased to 7.400000000000001 after 10 coins
Speed increased to 8.8 after 15 coins
Speed increased to 10.399999999999999 after 20 coins


SystemExit: 

C:\Users\ZEON\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import pygame
import random
import time

# Initialize pygame
pygame.init()

# Constants
GRID_SIZE = 20
snake_col = (0, 255, 0)
food_col = [(255, 0, 0), (255, 165, 0), (255, 255, 0)]  # Red, Orange, Yellow
bg = (0, 0, 0)
txt_col = (255, 255, 255)
eda_timer = 5000  # 5 seconds in milliseconds

# Set up display
screen = pygame.display.set_mode((600, 400))
pygame.display.set_caption("Enhanced Snake Game")

# Fonts
font = pygame.font.SysFont("Verdana", 20)

class Food:
    def __init__(self):
        self.position = self.generate_position()
        self.weight = random.randint(1, 3)
        self.color = food_col[self.weight-1]
        self.spawn_time = pygame.time.get_ticks()  # Record when food was spawned
    
    def generate_position(self):
        while True:
            pos = (random.randint(0, (600 // GRID_SIZE) - 1) * GRID_SIZE,
                  random.randint(0, (400 // GRID_SIZE) - 1) * GRID_SIZE)
            return pos
    
    def is_expired(self):
        current_time = pygame.time.get_ticks()
        return (current_time - self.spawn_time) > eda_timer

# Snake Initialization
snake = [(100, 100)]
snake_dir = (GRID_SIZE, 0)
foods = [Food()]
score = 0
level = 1
speed = 10

running = True
clock = pygame.time.Clock()

while running:
    screen.fill(bg)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP and snake_dir != (0, GRID_SIZE):
                snake_dir = (0, -GRID_SIZE)
            elif event.key == pygame.K_DOWN and snake_dir != (0, -GRID_SIZE):
                snake_dir = (0, GRID_SIZE)
            elif event.key == pygame.K_LEFT and snake_dir != (GRID_SIZE, 0):
                snake_dir = (-GRID_SIZE, 0)
            elif event.key == pygame.K_RIGHT and snake_dir != (-GRID_SIZE, 0):
                snake_dir = (GRID_SIZE, 0)
    
    # Move the snake
    new_head = (snake[0][0] + snake_dir[0], snake[0][1] + snake_dir[1])
    
    # Check for wall collision
    if new_head[0] < 0 or new_head[0] >= 600 or new_head[1] < 0 or new_head[1] >= 400:
        running = False
    
    # Check for self collision
    if new_head in snake:
        running = False
    
    # Add new head to the snake
    snake.insert(0, new_head)
    
    # Check for food collision
    food_eaten = None
    for food in foods:
        if new_head == food.position:
            score += food.weight  # Increase score based on food weight
            food_eaten = food
            
            # Increase level every 5 points
            if score % 5 == 0:
                level += 1
                speed += 2  # Increase speed
            break
    
    if food_eaten:
        foods.remove(food_eaten)
        foods.append(Food())  # Add new food
    else:
        snake.pop()  # Remove tail if no food eaten
    
    # Remove expired foods
    current_time = pygame.time.get_ticks()
    foods = [food for food in foods if not food.is_expired()]
    
    # Ensure we always have at least one food on screen
    if len(foods) < 1:
        foods.append(Food())
    
    # Draw all foods
    for food in foods:
        # Calculate remaining time percentage (for visual effect)
        time_left = 1 - min(1, (current_time - food.spawn_time) / eda_timer)
        # Draw food with fading effect based on remaining time
        color = tuple(int(c * time_left) for c in food.color)
        pygame.draw.rect(screen, color, (food.position[0], food.position[1], GRID_SIZE, GRID_SIZE))
    
    # Draw snake
    for segment in snake:
        pygame.draw.rect(screen, snake_col, (segment[0], segment[1], GRID_SIZE, GRID_SIZE))
    
    # Display score and level
    score_text = font.render(f"Score: {score}", True, txt_col)
    level_text = font.render(f"Level: {level}", True, txt_col)
    screen.blit(score_text, (10, 10))
    screen.blit(level_text, (10, 30))
    
    pygame.display.update()
    clock.tick(speed)

pygame.quit()

In [10]:
import pygame
import math

# Initialize pygame
pygame.init()
screen = pygame.display.set_mode((640, 480))
clock = pygame.time.Clock()

# Settings
mode = 'blue'
tool = 'pen'  # Default tool
shapes = []  # Store all drawn elements
poss = []  # Store pen points
eraser_path = []  # Store eraser points
ss = 50  # Fixed size for all shapes

def colorr():
    if mode == 'blue':
        return (0, 0, 255)
    elif mode == 'red':
        return (255, 0, 0)
    elif mode == 'green':
        return (0, 255, 0)

def draww():
    color = colorr()
    if tool != 'eraser' and len(poss) > 1:
        pygame.draw.lines(screen, color, False, poss, 2)
    
    if len(eraser_path) > 1:
        pygame.draw.lines(screen, (0, 0, 0), False, eraser_path, 10)
    
    for shape in shapes:
        tool_type, pos = shape
        if tool_type == 'circle':
            pygame.draw.circle(screen, color, pos, ss//2)
        elif tool_type == 'square':
            rect = pygame.Rect(pos[0]-ss//2, pos[1]-ss//2, ss, ss)
            pygame.draw.rect(screen, color, rect)
        elif tool_type == 'right':
            points = [
                (pos[0] - ss//2, pos[1] + ss//2),
                (pos[0] - ss//2, pos[1] - ss//2),
                (pos[0] + ss//2, pos[1] + ss//2)
            ]
            pygame.draw.polygon(screen, color, points)
        elif tool_type == 'equilateral':
            height = int(ss * math.sqrt(3)/2)
            points = [
                (pos[0], pos[1]-height//2),
                (pos[0]-ss//2, pos[1]+height//2),
                (pos[0]+ss//2, pos[1]+height//2)
            ]
            pygame.draw.polygon(screen, color, points)
        elif tool_type == 'rhombus':
            points = [
                (pos[0], pos[1]-ss//2),
                (pos[0]+ss//2, pos[1]),
                (pos[0], pos[1]+ss//2),
                (pos[0]-ss//2, pos[1])
            ]
            pygame.draw.polygon(screen, color, points)

def handle_events():
    global tool, mode, poss, eraser_path

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            return False
        
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                mode = 'red'
            elif event.key == pygame.K_g:
                mode = 'green'
            elif event.key == pygame.K_b:
                mode = 'blue'
            elif event.key == pygame.K_p:
                tool = 'pen'
                poss = []
            elif event.key == pygame.K_e:
                tool = 'eraser'
                eraser_path = []
            elif event.key == pygame.K_c:
                tool = 'circle'
            elif event.key == pygame.K_s:
                tool = 'square'
            elif event.key == pygame.K_t:
                tool = 'right'
            elif event.key == pygame.K_y:
                tool = 'equilateral'
            elif event.key == pygame.K_h:
                tool = 'rhombus'
        
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            if tool == 'pen':
                poss.append(event.pos)
            elif tool == 'eraser':
                eraser_path.append(event.pos)
            else:
                shapes.append((tool, event.pos))
        
        elif event.type == pygame.MOUSEMOTION:
            if pygame.mouse.get_pressed()[0]:
                if tool == 'pen':
                    poss.append(event.pos)
                elif tool == 'eraser':
                    eraser_path.append(event.pos)

    return True

running = True
while running:
    running = handle_events()
    screen.fill((0, 0, 0))
    draww()
    pygame.display.flip()
    clock.tick(60)

pygame.quit()
